# Rupture distances (Rrup, Rx, Rjb) in openquake.hazardlib

<tt>

LICENSE

Copyright (c) 2014, GEM Foundation, G. Weatherill, M. Pagani, D. Monelli.

The notebook is free software: you can redistribute
it and/or modify it under the terms of the GNU Affero General Public
License as published by the Free Software Foundation, either version
3 of the License, or (at your option) any later version.

You should have received a copy of the GNU Affero General Public License
along with OpenQuake. If not, see <http://www.gnu.org/licenses/>

DISCLAIMER
 
The notebook provided herein is released as a prototype
implementation on behalf of scientists and engineers working within the GEM
Foundation (Global Earthquake Model).

It is distributed for the purpose of open collaboration and in the
hope that it will be useful to the scientific, engineering, disaster
risk and software design communities.

The software is NOT distributed as part of GEM's OpenQuake suite
(http://www.globalquakemodel.org/openquake) and must be considered as a
separate entity. The software provided herein is designed and implemented
by scientific staff. It is not developed to the design standards, nor
subject to same level of critical review by professional software
developers, as GEM's OpenQuake software suite.

Feedback and contribution to the software is welcome, and can be
directed to the hazard scientific staff of the GEM Model Facility
(hazard@globalquakemodel.org).

The notebook is therefore distributed WITHOUT ANY WARRANTY; without
even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR
PURPOSE. See the GNU General Public License for more details.

The GEM Foundation, and the authors of the software, assume no liability for
use of the software.
</tt>

In [ ]:
%load_ext autoreload
%autoreload 2
import warnings; warnings.filterwarnings("ignore")

In [ ]:
%matplotlib inline

from openquake.hazardlib.geo import (
    Point, Line, PlanarSurface, MultiSurface, SimpleFaultSurface,
    ComplexFaultSurface, RectangularMesh
)

import os
import numpy
from matplotlib import pyplot
from mpl_toolkits.basemap import Basemap
from IPython.display import Image
from openquake.hazardlib.geo.mesh import Mesh

In [ ]:
def get_mesh_boundary(mesh):
    """
    Return coordinates of mesh boundary
    """
    boundary_lons = numpy.concatenate((mesh.lons[0, :], mesh.lons[1:, -1], mesh.lons[-1,:-1][::-1], mesh.lons[:-1, 0][::-1]))
    boundary_lats = numpy.concatenate((mesh.lats[0, :], mesh.lats[1:, -1], mesh.lats[-1,:-1][::-1], mesh.lats[:-1, 0][::-1]))
    
    return boundary_lons, boundary_lats

In [ ]:
def get_planar_surface_boundary(surf):
    """
    Return coordinates of planar surface boundary
    """
    boundary_lons = numpy.array(
        [surf.top_left.longitude, surf.top_right.longitude,
         surf.bottom_right.longitude, surf.bottom_left.longitude, surf.top_left.longitude]
    )
    boundary_lats = numpy.array(
        [surf.top_left.latitude, surf.top_right.latitude,
         surf.bottom_right.latitude, surf.bottom_left.latitude, surf.top_left.latitude]
    )
    
    return boundary_lons, boundary_lats

In [ ]:
def get_mesh(surf, buf=0.3, delta=0.001):
    """
    Return grid of nodes and map projection specific to surface
    """
    min_lon, max_lon, max_lat, min_lat = surf.get_bounding_box()
    
    min_lon -= buf
    max_lon += buf
    min_lat -= buf
    max_lat += buf

    lons = numpy.arange(min_lon, max_lon + delta, delta)
    lats = numpy.arange(min_lat, max_lat + delta, delta)
    lons, lats = numpy.meshgrid(lons, lats)
    mesh = RectangularMesh(lons=lons, lats=lats, depths=None)
    
    return mesh

In [ ]:
def write_rupture_file(surf, fname):
    fou = open(fname, 'w')   
    if hasattr(surf, 'top_left'):
        lons, lats = get_planar_surface_boundary(surf)
        fou.write('>\n')
        for lo, la in zip(lons, lats):
            fou.write('{:.4f} {:.4f}\n'.format(lo, la))
    elif hasattr(surf, 'surfaces'):
        for srf in surf.surfaces:
            lons, lats = get_planar_surface_boundary(srf)
            fou.write('>\n')
            for lo, la in zip(lons, lats):
                fou.write('{:.4f} {:.4f}\n'.format(lo, la))
    else:
        lons, lats = get_mesh_boundary(surf.mesh)
        fou.write('>\n')
        for lo, la in zip(lons, lats):
            fou.write('{:.4f} {:.4f}\n'.format(lo, la))
    fou.close()
    
def write_distance_file(dst, mesh, fname):
    shp = mesh.lons.size
    lo = mesh.lons.reshape(shp)
    la = mesh.lats.reshape(shp)
    ds = dst.reshape(shp)
    aa = numpy.vstack((lo, la, ds)).T
    numpy.savetxt(fname, aa)

# Distances to single Planar Surface

In [ ]:
# define planar surface
surf= PlanarSurface(
    strike=3., dip=40.,
    top_left = Point(depth=2.0, latitude=36.539, longitude=137.874),
    top_right = Point(depth=2.0, latitude=36.7680117889, longitude=137.888982562),
    bottom_left = Point(depth=14.8557521937, latitude=36.5316665297, longitude=138.045238973),
    bottom_right = Point(depth=14.8557521937, latitude=36.7606772927, longitude=138.060731463)
)
# get grid of points for which to calculate distances and associated map projection
mesh = get_mesh(surf)
# compute and plot Rrup
dst = surf.get_min_distance(mesh)

In [ ]:
fname = '/tmp/ruptures.xyz'
fname_dst = '/tmp/distances.xyz'
figname = '/tmp/planar_rupture_distances.ps'
write_rupture_file(surf, fname)
write_distance_file(dst, mesh, fname_dst)
!./plot_distance_field.sh "$fname" "$fname_dst" "$figname" 10
Image(filename=os.path.splitext(figname)[0]+'.png', width=400)

In [ ]:
# compute and plot Rjb
dst = surf.get_joyner_boore_distance(mesh)

In [ ]:
fname = '/tmp/ruptures.xyz'
fname_dst = '/tmp/distances.xyz'
figname = '/tmp/planar_rupture_distances.ps'
write_rupture_file(surf, fname)
write_distance_file(dst, mesh, fname_dst)
!./plot_distance_field.sh "$fname" "$fname_dst" "$figname" 10
Image(filename=os.path.splitext(figname)[0]+'.png', width=400)

In [ ]:
# compute and plot Rx
dst = surf.get_rx_distance(mesh)

In [ ]:
fname = '/tmp/ruptures.xyz'
fname_dst = '/tmp/distances.xyz'
figname = '/tmp/planar_rupture_distances.ps'
write_rupture_file(surf, fname)
write_distance_file(dst, mesh, fname_dst)
!./plot_distance_field.sh "$fname" "$fname_dst" "$figname" 10
Image(filename=os.path.splitext(figname)[0]+'.png', width=400)

# Distances to Simple Fault Surface

In [ ]:
# create Simple Fault Surface
surf = SimpleFaultSurface.from_fault_data(
    fault_trace=Line([Point(9.21602706445, 45.1555287905), Point(9.25645636929, 45.1877167851),
                  Point(9.29688464252, 45.2199047798), Point(9.35715705075, 45.2398017764),
                  Point(9.42902686305, 45.2401237764), Point(9.47246500782, 45.2381597767),
                  Point(9.51590215304, 45.236194777), Point(9.56736930079, 45.2307927779),
                  Point(9.61883544823, 45.2253897788), Point(9.67030259419, 45.2199877797),
                  Point(9.72270625188, 45.2033947825), Point(9.77510990175, 45.1868007853),
                  Point(9.83238881096, 45.1680237884), Point(9.88966771001, 45.1492457915),
                  Point(9.94694559775, 45.1304687947), Point(10.0042244753, 45.1116907978)]),
    upper_seismogenic_depth=2.,
    lower_seismogenic_depth=7.,
    dip=30.,
    mesh_spacing=4.
)
# get grid of points for which to calculate distances and associated map projection
mesh = get_mesh(surf, delta=0.002)
# compute and plot Rrup
dst = surf.get_min_distance(mesh)

In [ ]:
fname = '/tmp/ruptures.xyz'
fname_dst = '/tmp/distances.xyz'
figname = '/tmp/planar_rupture_distances.ps'
write_rupture_file(surf, fname)
write_distance_file(dst, mesh, fname_dst)
!./plot_distance_field.sh "$fname" "$fname_dst" "$figname" 10
Image(filename=os.path.splitext(figname)[0]+'.png', width=500)

In [ ]:
# compute and plot Rjb
shp = mesh.lons.size
lons = mesh.lons.reshape(shp)
lats = mesh.lats.reshape(shp)
mesh_new = Mesh(lons, lats)
dst = surf.get_joyner_boore_distance(mesh_new)

In [ ]:
fname = '/tmp/ruptures.xyz'
fname_dst = '/tmp/distances.xyz'
figname = '/tmp/planar_rupture_distances.ps'
write_rupture_file(surf, fname)
write_distance_file(dst, mesh, fname_dst)
!./plot_distance_field.sh "$fname" "$fname_dst" "$figname" 10
Image(filename=os.path.splitext(figname)[0]+'.png', width=500)

# Distances to Complex Fault Surface

In [ ]:
top_edge = Line([Point(144.069555556, 39.7059999996, 9.202), Point(143.987666642, 39.0726212093, 9.202),
                 Point(143.844804987, 38.4446225095, 9.202), Point(143.715121187, 37.8147879271, 9.202),
                 Point(143.452451355, 37.2149977193, 9.202), Point(143.086155668, 36.6503300815, 9.202),
                 Point(142.589773574, 36.1569466634, 9.202), Point(142.202909091, 35.606, 9.202)])

bottom_edge = Line([Point(142.0450625, 39.7059999997, 48.202), Point(141.970993342, 39.1530522533, 48.202),
                    Point(141.893707006, 38.6004781864, 48.202), Point(141.734990146, 38.0590949727, 48.202),
                    Point(141.52384879, 37.5292617122, 48.202), Point(141.2114751, 37.0322180634, 48.202),
                    Point(140.900140283, 36.5353773475, 48.202), Point(140.697696296, 36.006, 48.202)])

edges = [top_edge, bottom_edge]

surf = ComplexFaultSurface.from_fault_data(edges, mesh_spacing=10.)

# get grid of points for which to calculate distances and associated map projection
mesh = get_mesh(surf, delta=0.05)

# compute and plot Rrup
dst = surf.get_min_distance(mesh)

In [ ]:
fname = '/tmp/ruptures.xyz'
fname_dst = '/tmp/distances.xyz'
figname = '/tmp/planar_rupture_distances.ps'
write_rupture_file(surf, fname)
write_distance_file(dst, mesh, fname_dst)
!./plot_distance_field.sh "$fname" "$fname_dst" "$figname" 2
Image(filename=os.path.splitext(figname)[0]+'.png', width=500)

In [ ]:
# compute and plot Rjb
shp = mesh.lons.size
lons = mesh.lons.reshape(shp)
lats = mesh.lats.reshape(shp)
mesh_new = Mesh(lons, lats)
dst = surf.get_joyner_boore_distance(mesh_new)

In [ ]:
fname = '/tmp/ruptures.xyz'
fname_dst = '/tmp/distances.xyz'
figname = '/tmp/planar_rupture_distances.ps'
write_rupture_file(surf, fname)
write_distance_file(dst, mesh, fname_dst)
!./plot_distance_field.sh "$fname" "$fname_dst" "$figname" 2
Image(filename=os.path.splitext(figname)[0]+'.png', width=500)

# Distances to Multi Surface

In [ ]:
# define Multi Surface as list of planar surfaces
p1 = PlanarSurface(
    strike=3., dip=40.,
    top_left = Point(depth=2.0, latitude=36.539, longitude=137.874),
    top_right = Point(depth=2.0, latitude=36.7680117889, longitude=137.888982562),
    bottom_left = Point(depth=14.8557521937, latitude=36.5316665297, longitude=138.045238973),
    bottom_right = Point(depth=14.8557521937, latitude=36.7606772927, longitude=138.060731463)
)
p2 = PlanarSurface(
    strike=344., dip=40.,
    top_left = Point(depth=2.0, latitude=36.2320586295, longitude=137.983095101),
    top_right = Point(depth=2.0, latitude=36.539, longitude=137.874),
    bottom_left = Point(depth=14.8557521937, latitude=36.2699248045, longitude=138.147372206),
    bottom_right = Point(depth=14.8557521937, latitude=36.5768649083, longitude=138.038927779)
)
p3 = PlanarSurface(
    strike=337., dip=80.,
    top_left = Point(depth=2.0, latitude=36.0836674599, longitude=138.039395087),
    top_right = Point(depth=2.0, latitude=36.212, longitude=137.972),
    bottom_left = Point(depth=15.7873085422, latitude=36.0922075079, longitude=138.064300307),
    bottom_right = Point(depth=15.7873085422, latitude=36.2205400358, longitude=137.996946016)
)
p4 = PlanarSurface(
    strike=318., dip=80.,
    top_left = Point(depth=2.0, latitude=35.8588534701, longitude=138.287735144),
    top_right = Point(depth=2.0, latitude=36.083, longitude=138.039),
    bottom_left = Point(depth=15.7873085422, latitude=35.8734811262, longitude=138.307786048),
    bottom_right = Point(depth=15.7873085422, latitude=36.0976276423, longitude=138.059107946)
)
surf = MultiSurface([p1, p2, p3, p4])

# get grid of points for which to calculate distances and associated map projection
mesh = get_mesh(surf)

# compute and plot Rrup
dst = surf.get_min_distance(mesh)

In [ ]:
fname = '/tmp/ruptures.xyz'
fname_dst = '/tmp/distances.xyz'
figname = '/tmp/planar_rupture_distances.ps'
write_rupture_file(surf, fname)
write_distance_file(dst, mesh, fname_dst)
!./plot_distance_field.sh "$fname" "$fname_dst" "$figname" 6
Image(filename=os.path.splitext(figname)[0]+'.png', width=400)

In [ ]:
# compute and plot Rjb
dst = surf.get_joyner_boore_distance(mesh)

In [ ]:
fname = '/tmp/ruptures.xyz'
fname_dst = '/tmp/distances.xyz'
figname = '/tmp/planar_rupture_distances.ps'
write_rupture_file(surf, fname)
write_distance_file(dst, mesh, fname_dst)
!./plot_distance_field.sh "$fname" "$fname_dst" "$figname" 6
Image(filename=os.path.splitext(figname)[0]+'.png', width=400)

In [ ]:
# compute and plot Rx
dst = surf.get_rx_distance(mesh)

In [ ]:
fname = '/tmp/ruptures.xyz'
fname_dst = '/tmp/distances.xyz'
figname = '/tmp/planar_rupture_distances.ps'
write_rupture_file(surf, fname)
write_distance_file(dst, mesh, fname_dst)
!./plot_distance_field.sh "$fname" "$fname_dst" "$figname" 6
Image(filename=os.path.splitext(figname)[0]+'.png', width=400)